In [4]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
sys.path.append("../")
import zipfile
import urllib.request
from shutil import copyfile

In [18]:
# Go to the input folder
def change_dir_to_input(newpath):
    get_path = sys.path[0].split("\\")      
    del get_path[-1]                        
    cwd = "input\\" + str(newpath)                          
    get_path.append(cwd)                    
    path = "\\".join(get_path)              
    os.chdir(path)                         

In [20]:
# Get input files from 2014 to 2020
for i in range(14,21):
    change_dir_to_input('occupation_orig')
    url = 'https://www.bls.gov/oes/special.requests/oesm'+ str(i) + 'ma.zip'
    urllib.request.urlretrieve(url, 'oesm'+ str(i) + 'ma.zip')
    zf = zipfile.ZipFile('oesm'+ str(i) + 'ma.zip') 
    df = pd.read_excel(zf.open('oesm'+ str(i) + 'ma/MSA_M20'+ str(i) +'_dl.xlsx'))
    df.columns = map(str.upper, df.columns)
    if 'OCC_GROUP' in list(df.columns):
        df = df.rename(columns={'OCC_GROUP': 'O_GROUP'})
    df = df[df.O_GROUP.str.contains("major")]
    change_dir_to_input('occupation_out')
    df.to_csv('MSA20'+str(i)+'.csv')